In [ ]:
import logging

import numpy

import cicada.additive
import cicada.communicator

logging.basicConfig(level=logging.INFO)

@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    # Player 0 will provide the secret divisor.
    a = numpy.array(144409)
    # Every player knows the modulus.
    for i in range(10):
        b = numpy.array(117+i)

        log.info(f"Dividend: {a}", src=0)
        log.info(f"Modulus: {b}", src=1)

        a_share = protocol.share(src=0, secret=protocol.encoder.encode(a), shape=())
        remainder_share = protocol.private_public_mod(a_share, b)
        remainder = protocol.encoder.decode(protocol.reveal(remainder_share))

        log.info(f"Player {communicator.rank} remainder: {remainder} expected {a%b}", src=0)

main();

As you can see, the result revealed to the players is correct.